In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3
from glob import glob

In [3]:
data_dir = "/kaggle/input/dataset/dataset_dr"
batch_size =32
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    seed=1,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    data_dir, # same directory as training data
    target_size=(227, 227),
    batch_size=batch_size,
    seed=1,
    class_mode='categorical',
    subset='validation') # set as validation data

In [2]:
IncV3 = InceptionV3(include_top = False, weights = "imagenet",input_shape = (227,227,3))
model = Sequential()

model.add(IncV3)
model.add(Flatten())

model.add(Dense(units = 2048, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(units = 5, activation = "softmax"))

In [8]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [4]:
model = keras.models.load_model('/kaggle/input/modelv1/model.h5')

In [5]:
nb_epochs = 100
model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

In [6]:
model.save('model.h5')

In [7]:
x=np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
#y=np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
print(x.shape)
#print(y.shape)

In [8]:
x_val=np.concatenate([validation_generator.next()[0] for i in range(validation_generator.__len__())])
y_val=np.concatenate([validation_generator.next()[1] for i in range(validation_generator.__len__())])
print(x_val.shape)
print(y_val.shape)

In [9]:
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
def build_autoencoder(img_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(np.prod(img_shape))) # np.prod(img_shape) is the same as 32*32*3, it's more generic than saying 3072
    decoder.add(Reshape(img_shape))

    return encoder, decoder
IMG_SHAPE = x.shape[1:]
encoder, decoder = build_autoencoder(IMG_SHAPE, 64)

inp = Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

EPOCHS = 20
INIT_LR = 1e-3
BS = 32
autoencoder = Model(inp,reconstruction)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(optimizer=opt, loss='mse')

print(autoencoder.summary())


In [10]:
autoencoder.fit(x=x, y=x,  epochs=25,
                batch_size=BS,
                validation_data=(x_val, x_val))

In [8]:
#RAM issues with Kaggle
del x

In [9]:
x_pert = np.load('/kaggle/input/dataset/perturbed_images.npy')

In [10]:
x_val = np.vstack((x_val,x_pert)) 

In [11]:
x_val.shape

In [12]:
decoded = autoencoder.predict(x_val)
errors = []
# loop over all original images and their corresponding
# reconstructions
for (image, recon) in zip(x_val, decoded):
    # compute the mean squared error between the ground-truth image
    # and the reconstructed image, then add it to our list of errors
    mse = np.mean((image - recon) ** 2)
    errors.append(mse)

In [13]:
c=0
for e in errors:
    if e > 0.05:
        c+=1
print(c)
print((c/731)*100)


In [13]:
import seaborn as sns
sns.set_style('darkgrid') 
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

In [14]:
x1 = [i for i in range(0,732)]
x2 = [i for i in range(732,1463)]
plt.plot(x1, errors[:732], 'o', color='green');
print(len(errors))
plt.plot(x2, errors[732:], 'o', color='red');
plt.xlabel('Index of the Image')
plt.ylabel('Reconstructed Error')
plt.legend(['Original','Perturbed'])
plt.title("Reconstruction errors of the test set")
plt.savefig('Auto_RE.png')
plt.show()

In [40]:
# From the plot above we can set the threshold to 0.05 and say the images
# that are having loss value more than that are perturbed.

In [1]:
import matplotlib.pyplot as plt
plt.imshow(x_val[731])

In [2]:
autoencoder.evaluate(x_pert,x_pert)

In [3]:
autoencoder.evaluate(x_val,x_val)